In [2]:
import spoti
from requests import get, post
import json
import webbrowser
import requests
import base64
import os
import urllib.parse
import pandas as pd
import time

In [3]:
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')
redirect_uri = "http://localhost:8501/callback"

api_token = spoti.get_token()
scope = ''

In [4]:
def get_authorization_code():
    auth_url = (
        "https://accounts.spotify.com/authorize"
        "?response_type=code"
        f"&client_id={client_id}"
        f"&scope={scope}"
        f"&redirect_uri={urllib.parse.quote(redirect_uri, safe='')}"
    )

    webbrowser.open(auth_url)

    return input("Paste the full redirect URL here: ")


def extract_auth_code(redirect_url):
    parsed_url = urllib.parse.urlparse(redirect_url)
    auth_code = urllib.parse.parse_qs(parsed_url.query).get("code", [None])[0]
    
    return auth_code


def get_access_token(auth_code):
    token_url = "https://accounts.spotify.com/api/token"
    auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

    headers = {
        "Authorization": f"Basic {auth_header}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": redirect_uri
    }

    response = requests.post(token_url, headers=headers, data=data)
    response_json = response.json()

    return response_json["access_token"]


In [5]:
def save_to_csv(recent_tracks, filename):
      
    for track in recent_tracks:
        track["artist_names"] = ", ".join([artist["artist_name"] for artist in track["artists"]])
        track["artist_ids"] = ", ".join([artist["artist_id"] for artist in track["artists"]])
        del track["artists"]
    
    df = pd.DataFrame(recent_tracks)
    df.to_csv(filename, index=False, encoding='utf-8')
    
    print('Data saved to CSV file')

### Make Database of Listening History from Recents

In [12]:
def recently_played(access_token):
    url = "https://api.spotify.com/v1/me/player/recently-played?limit=50"
    headers = {"Authorization": f"Bearer {access_token}"}

    result = get(url, headers=headers)
    json_result = json.loads(result.content)
    
    track_info = []
    
    for item in json_result["items"]:
        
        track = item["track"]
        info = {
            "track_name": track["name"],
            "track_id": track["id"],
            "album_name": track["album"]["name"],
            "album_id": track["album"]["id"],
            "album_release_date": track["album"]["release_date"],
            "popularity": track["popularity"],
            "artists": [
                {"artist_name": artist["name"], "artist_id": artist["id"]}
                for artist in track["artists"]
            ],
        }
        
        track_info.append(info)
    
    return track_info


scope = "user-read-recently-played"

redirect_url = get_authorization_code()
auth_code = extract_auth_code(redirect_url)
access_token = get_access_token(auth_code)
print(f"✅ Access Token")

recent_tracks = recently_played(access_token)
save_to_csv(recent_tracks, filename="new_recents.csv")

✅ Access Token
Data saved to CSV file


In [13]:
full_recents = pd.read_csv('recent_tracks.csv', encoding='utf-8')
new_recents = pd.read_csv('new_recents.csv', encoding='utf-8')

recents_df = pd.concat([full_recents, new_recents], ignore_index=True)
recents_df.drop_duplicates(keep='last', inplace=True)
recents_df.to_csv('recent_tracks.csv', index=False, encoding='utf-8')

### Fetch All Liked Songs

In [ ]:
def liked_songs(token):
    
    url = 'https://api.spotify.com/v1/me/tracks?limit=50'
    headers = {'Authorization': f'Bearer {token}'}
    
    all_likes = []
    
    while url:
        
        result = get(url, headers=headers)
        json_result = json.loads(result.content)
        
        for item in json_result["items"]:
            
            track = item["track"]
            info = {
                "track_name": track["name"],
                "track_id": track["id"],
                "album_name": track["album"]["name"],
                "album_id": track["album"]["id"],
                "album_release_date": track["album"]["release_date"],
                "popularity": track["popularity"],
                "artists": [
                    {"artist_name": artist["name"], "artist_id": artist["id"]}
                    for artist in track["artists"]
                ],
            }
            all_likes.append(info)
            
        url = json_result["next"]
    
    return all_likes


scope = "user-library-read"

redirect_url = get_authorization_code()
auth_code = extract_auth_code(redirect_url)
access_token = get_access_token(auth_code)
print(f"✅ Access Token: {access_token}")

liked_songs = liked_songs(access_token)
# save_to_csv(liked_songs, filename="liked_songs.csv")

### Create Database of Liked Songs per Album

In [17]:
df = pd.read_csv('liked_songs.csv')
album_wise_liked = []

for album in df['album_name'].unique():
    liked_album_data = df.loc[df['album_name']==album]
    
    liked_popularity = liked_album_data['popularity'].mean()
    album_liked = ""
    
    for song in liked_album_data['track_name']:
        album_liked += song + ', '
        
    entry = {
        "album_name": album,
        "liked_songs": album_liked.rstrip(', '),
        "liked_popularity": round(liked_popularity, 2)
    }
    album_wise_liked.append(entry)
    
album_wise_liked = pd.DataFrame(album_wise_liked)
album_wise_liked.to_csv('album_wise_liked.csv')
album_wise_liked.head(10)

,album_name,liked_songs,liked_popularity
0,Jinx,Ghostride,72.00
1,MUSIC,"OVERLY, HBA, WALK, OPM BABI, OLYMPIAN, COCAINE...",83.81
2,AT.LONG.LAST.A$AP,"M'$ (feat. Lil Wayne), Better Things, Wavybone...",63.08
3,Sensational,"1 Thot 2 Thot Red Thot Blue Thot, Whip A Tesla...",50.67
4,Yasuke,"Between Memories, The Eyes of Vengeance, Your ...",33.14
5,Arbi Ghode,Arbi Ghode,57.00
6,"Mancini Plays the Theme from ""Love Story""","Theme from ""Love Story""",34.00
7,Make Way for Dionne Warwick,Walk on By,67.00
8,Evr’y Time We Say Goodbye - All-Time Romantic ...,Georgia on My Mind,56.00
9,The Genesis of Aretha: 1960-1966,One Step Ahead,47.00
